# Tune Tutorial

<img src="tune.png" alt="Tune Logo" width="400"/>


Tune is a scalable framework for model training and hyperparameter search with a focus on deep learning and deep reinforcement learning.

**Code**: https://github.com/ray-project/ray/tree/master/python/ray/tune

**Examples**: https://github.com/ray-project/ray/tree/master/python/ray/tune/examples

**Documentation**: http://ray.readthedocs.io/en/latest/tune.html

**Mailing List** https://groups.google.com/forum/#!forum/ray-dev

# Overview

Tuning hyperparameters is often the most expensive part of the machine learning workflow. Tune is built to address this, demonstrating an efficient and scalable solution for this pain point.

This tutorial will walk you through the following process:

1. Creating and training a model on a toy dataset (MNIST)
2. Integrating Tune into your workflow by creating a trainable and running an experiment
3. Trying out advanced features - plugging in an efficient scheduler
4. (Optional) Try out a search algorithm

## Part 1: Creating and training an un-Tune-d model

In [ ]:
from IPython.display import HTML


from model import load_data, make_model, evaluate
from helper import prepare_data, limit_threads

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
limit_threads(4)

Let's create and train a model to classify [MNIST](https://www.wikiwand.com/en/MNIST_database) digits **without tuning**, as a baseline first model. 

We will be creating a Convolutional Neural Network model (using [Keras](https://keras.io/)) to classify the digits. 

In [ ]:
def train_mnist():
    x_train, y_train, x_test, y_test = load_data()
    model = make_model(lr=0.1, layer_size=4)
    model.fit(x_train, y_train, verbose=1, batch_size=256)
    return model

Let's create our model. (This should take ~10 seconds)

In [ ]:
model = train_mnist()

Lets evaluate the un-Tune-d model.

In [ ]:
evaluate(model)

Let's now quickly try out this model to see if it works as expected.

We'll load the model with our trained weights. 

Try to write a digit into the box below - *do not rerun the cell after you write the digit*.

In [ ]:
data = None
HTML(open("input.html").read())

(tip: don't expect it to work)

In [ ]:
prepared_data = prepare_data(data)
print("This model predicted your input as", model.predict(prepared_data).argmax())

## Part 2: Setting up Tune

In [ ]:
import numpy as np

import ray
from ray import tune

from helper import test_reporter, get_best_result, get_best_model
from model import load_data, make_model

One thing we might want to do now is find better hyperparameters so that our model trains more quickly and possibly to a higher accuracy. Let's make some minor modifications to utilize Tune. 

### Step 1: Defining a Trainable to run

Tune will automate and distribute your hyperparameter search with multiple **trials**. Each trial runs a user-defined Python function called a **trainable**. 

We define a new training function ``train_mnist_tune`` as our Trainable. A trainable function must:

 1. pass in a `tune_reporter` object like the following: ``def train_mnist_tune(config, tune_reporter)``
 2. train the model
 3. report some metric(s) to Tune. 

Step (3) allows Tune to keep track of performance as the model is training. 

Tune works seamlessly with many frameworks, and in this example, we'll use a custom Keras Callback ``TuneCallback`` object to report metrics.


The custom callback does the following on each batch:

```python
def on_batch_end(self, batch, logs={}):
    """Reports the last result and checkpoints if necessary."""
    ...
    self.tune_reporter(
        mean_accuracy=np.mean(self.last_10_results), 
        checkpoint="weights_tune.h5")
```

In [ ]:
import keras
import os

from helper import TuneCallback

In [ ]:
# FIXME: add an argument here so that it takes in reporter
def train_mnist_tune(config):  
#########################################################
    x_train, y_train, _, _ = load_data()
    model = make_model(lr=config.get("lr", 0.1), layer_size=config.get("layer_size", 4))
    callbacks = [TuneCallback(tune_reporter)]
    for i in range(10):
        model.fit(x_train, y_train, verbose=0, batch_size=256, callbacks=callbacks)

In [ ]:
# This may take 30 seconds or so to run if incorrectly written
assert test_reporter(train_mnist_tune)

*Note: Call ``help(tune.Trainable)`` if you are interested in learning more about what qualifies as trainable in Tune.*

In [ ]:
help(tune.Trainable)

### Step 2: Configure the search and run Tune

Now that we have a working trainable, we want to use Tune to train it. We will use some basic Tune features for training: specifying a stopping criteria and a search space. 

There are two steps in this section. **TODO**: Configure ``tune.run`` with a stopping criteria using ``stop`` and a search space using ``config``.


1) Set the stopping criteria to when ``mean_accuracy`` passes `0.8` and when ``training_iteration`` passes 50.

For example, to specify that trials will be stopped whenever they report a `mean_accuracy` that is `>= 0.8`, do:

```python
stop={"mean_accuracy": 0.8}
```


2) We also want to designate a search space. We'll search over *learning rate*, which sets the step size of our model update, and *layer_size*, which corresponds
to the size of the last layer of our convolutional neural network.

For `layer_size`, Tune supports sampling parameters from user-specified lambda functions, which can be used independently or in combination with grid search. For learning rate, you can use `tune.grid_search` to specify an axis of a grid search. For example:

```python
space = {
    "lr": tune.grid_search([0.001, 0.05, 0.1]),
    "layer_size": tune.sample_from(lambda spec: int(np.random.uniform(16, 256))),
}
```


When you're ready, run the experiment! (this should take ~1 minute)

In [ ]:
# Feel free to ignore this; this is a workaround to help Ray deal with interrupts.
ray.shutdown(); ray.init(ignore_reinit_error=True)
#################################################


# !! FIXME: Fix me with the instructions above
space = {}
assert "layer_size" in space
assert "lr" in space

stop = {}
assert "training_iteration" in stop
assert "mean_accuracy" in stop
###############################################


trials = tune.run(
    train_mnist_tune,
    stop=stop, 
    config=space,
    resources_per_trial={"cpu": 2},
    verbose=1
)

You can expect the result below to be about `0.6`, although your mileage may vary (and it's OK).

In [ ]:
print("The best result is", get_best_result(trials, metric="mean_accuracy"))

As you can see, the accuracy is still low, similar to the accuracy of our first un-Tune-d model! In the next section, we will scale up the search and accelerate the training using a state of the art algorithm.

*Note: Call ``help(tune.run)`` if you are interested in learning more about executing experiments.*

## Part 3: Scale up the search with more samples, hyperparameters, and a custom scheduler 

`Hyperband` is state of the art algorithm that is used to early terminate un-promising trials. It has been shown to beat many state of the art hyperparameter optimization techniques and algorithms across a wide variety of datasets. **Tune allows users to take advantage this powerful algorithm in a couple lines of code.**

See https://blog.ml.cmu.edu/2018/12/12/massively-parallel-hyperparameter-optimization/ for more information.

1) Create an Asynchronous HyperBand Scheduler (ASHA).
```python
from ray.tune.schedulers import AsyncHyperBandScheduler

custom_scheduler = AsyncHyperBandScheduler(
    reward_attr='mean_accuracy',
    grace_period=5,
)
```

*Note: Read the documentation on this step at https://ray.readthedocs.io/en/latest/tune-schedulers.html#asynchronous-hyperband or call ``help(tune.schedulers.AsyncHyperBandScheduler)`` to learn more about the Asynchronous Hyperband Scheduler*

2) With this, we can afford to **increase the search space by 5x**. To do this, set the parameter `num_samples`. For example,

```python
tune.run(... num_samples=5)
```



When you're ready, run the experiment! (this should take ~1 minute)

In [ ]:
# Feel free to ignore this; this is a workaround to help Ray deal with interrupts.
ray.shutdown(); ray.init(ignore_reinit_error=True)
#################################################

# FIXME: Fix me with the instructions above
custom_scheduler = 
###############################################


better_trials = tune.run(
    train_mnist_tune,
    # FIXME: Fix me with the instructions above
    num_samples=0,  
    ###############################################
    scheduler=custom_scheduler,
    stop={"mean_accuracy": 0.95, "training_iteration": 50},  # The mean accuracy is increased now.
    config=space,  # We use the same search space as before.
    resources_per_trial={"cpu": 2},
    verbose=1,
    reuse_actors=True,  # This is an internal optimization to cache actors.
)

You can expect the result to be about `0.95`, although your mileage may vary.

In [ ]:
print("The best result is", get_best_result(better_trials, metric="mean_accuracy"))

Try out a newly tuned model.

In [ ]:
tuned_model = get_best_model(make_model, better_trials, "mean_accuracy")

In [ ]:
final_data = None
HTML(open("input_final.html").read())

In [ ]:
prepared_data = prepare_data(final_data)
print("This model predicted your input as", tuned_model.predict(prepared_data).argmax())

# 🎉 Congratulations, you're now a Tune expert! 🎉

# Please: fill out this form to provide feedback on this tutorial!

https://goo.gl/forms/NVTFjUKFz4TH8kgK2

# (Optional) Try using a search algorithm

Tune is an execution layer, so we can combine powerful optimizers such as HyperOpt (https://github.com/hyperopt/hyperopt) with state-of-the-art algorithms such as HyperBand without modifying any model training code.

The documentation to doing this is here: https://ray.readthedocs.io/en/latest/tune-searchalg.html#hyperopt-search-tree-structured-parzen-estimators

In [ ]:
from hyperopt import hp
from ray.tune.suggest.hyperopt import HyperOptSearch

space = {
    "lr": hp.uniform("lr", 0.001, 0.1),
    "momentum": hp.uniform("momentum", 0.1, 0.9),
    "hidden": hp.choice("hidden", np.arange(16, 256, dtype=int)),
}

hyperband = AsyncHyperBandScheduler(time_attr='timesteps_total', reward_attr='mean_accuracy')

hyperopt_search = HyperOptSearch(space, reward_attr="mean_accuracy")

good_results = tune.run(
    train_mnist_tune,
    num_samples=5,
    search_alg=hyperopt_search,
    scheduler=hyperband,
    stop={"mean_accuracy": 0.95},
    config=space,
    verbose=False
)

In [ ]:
print("The best result is", get_best_result(good_results, metric="mean_accuracy"))